<a href="https://colab.research.google.com/github/srujayreddyv/CSUS-CSC219-MachineLeaning/blob/main/labs/CSC219Lab02_Tensorflow_Srujay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CSC 219 Machine Learning (Fall 2023)

# Lab 2: Tensorflow Introduction

## 2.0 Helpful functions for Tensorflow (little gems)

The following functions will be used with TensorFlow to help preprocess the data.  They allow you to build the feature vector for a neural network.

* Predictors/Inputs
    * Fill any missing inputs with the median for that column.  Use **missing_median**.
    * Encode textual/categorical values with **encode_text_dummy**.
    * Encode numeric values with **encode_numeric_zscore**.
* Output
    * Discard rows with missing outputs.
    * Encode textual/categorical values with **encode_text_index**.
    * Do not encode output numeric values.
* Produce final feature vectors (x) and expected output (y) with **to_xy**.

In [ ]:
from collections.abc import Sequence
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)



# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low


## 2.1 Classification or Regression

Neural networks can function in *** classification or regression***:

* **Regression** - You expect a number as your neural network's prediction.
* **Classification** - You expect a class/category as your neural network's prediction.

Regression networks always have a single output neuron.  Classification neural networks have an output neuron for each class.

These neurons are grouped into layers:

* **Input Layer** - The input layer accepts feature vectors from the dataset.  Input layers usually have a bias neuron.
* **Output Layer** - The output from the neural network.  The output layer does not have a bias neuron.
* **Hidden Layers** - Layers that occur between the input and output layers.  Each hidden layer will usually have a bias neuron.


### What version of TensorFlow do you have?

TensorFlow is very new and changing rapidly.

In [ ]:
import tensorflow as tf
print("Tensor Flow Version: {}".format(tf.__version__))

Tensor Flow Version: 2.13.0


## 2.2 Example of Tensorflow Regression: MPG Prediction

This example shows how to encode the MPG dataset for regression.  Notice that:

* Input has both numeric and categorical
* Input has missing values

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
path = "/content/drive/MyDrive/Colab Notebooks/CSC219-MachineLearning/labs/data"

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

df[0:5]

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [ ]:
cars = df['name']

df.drop('name',1,inplace=True)

missing_median(df, 'horsepower')

encode_text_dummy(df, 'origin')

x,y = to_xy(df,"mpg")

<ipython-input-9-65707f89fc55>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop('name',1,inplace=True)


In [ ]:
x.shape

(398, 9)

In [ ]:
y.shape

(398,)

In [ ]:
x

array([[  8., 307., 130., ...,   1.,   0.,   0.],
       [  8., 350., 165., ...,   1.,   0.,   0.],
       [  8., 318., 150., ...,   1.,   0.,   0.],
       ...,
       [  4., 135.,  84., ...,   1.,   0.,   0.],
       [  4., 120.,  79., ...,   1.,   0.,   0.],
       [  4., 119.,  82., ...,   1.,   0.,   0.]], dtype=float32)

In [ ]:
y

array([18. , 15. , 18. , 16. , 17. , 15. , 14. , 14. , 14. , 15. , 15. ,
       14. , 15. , 14. , 24. , 22. , 18. , 21. , 27. , 26. , 25. , 24. ,
       25. , 26. , 21. , 10. , 10. , 11. ,  9. , 27. , 28. , 25. , 25. ,
       19. , 16. , 17. , 19. , 18. , 14. , 14. , 14. , 14. , 12. , 13. ,
       13. , 18. , 22. , 19. , 18. , 23. , 28. , 30. , 30. , 31. , 35. ,
       27. , 26. , 24. , 25. , 23. , 20. , 21. , 13. , 14. , 15. , 14. ,
       17. , 11. , 13. , 12. , 13. , 19. , 15. , 13. , 13. , 14. , 18. ,
       22. , 21. , 26. , 22. , 28. , 23. , 28. , 27. , 13. , 14. , 13. ,
       14. , 15. , 12. , 13. , 13. , 14. , 13. , 12. , 13. , 18. , 16. ,
       18. , 18. , 23. , 26. , 11. , 12. , 13. , 12. , 18. , 20. , 21. ,
       22. , 18. , 19. , 21. , 26. , 15. , 16. , 29. , 24. , 20. , 19. ,
       15. , 24. , 20. , 11. , 20. , 21. , 19. , 15. , 31. , 26. , 32. ,
       25. , 16. , 16. , 18. , 16. , 13. , 14. , 14. , 14. , 29. , 26. ,
       26. , 31. , 32. , 28. , 24. , 26. , 24. , 26

In [ ]:
model = Sequential()

model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1     #  why input_dim=x.shape[1]?
model.add(Dense(10, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output

model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(x,y,verbose=2,epochs=100)    # Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.


Epoch 1/100
13/13 - 1s - loss: 15389.0176 - 624ms/epoch - 48ms/step
Epoch 2/100
13/13 - 0s - loss: 3643.0308 - 14ms/epoch - 1ms/step
Epoch 3/100
13/13 - 0s - loss: 744.7773 - 15ms/epoch - 1ms/step
Epoch 4/100
13/13 - 0s - loss: 501.3780 - 16ms/epoch - 1ms/step
Epoch 5/100
13/13 - 0s - loss: 432.9582 - 15ms/epoch - 1ms/step
Epoch 6/100
13/13 - 0s - loss: 380.4862 - 14ms/epoch - 1ms/step
Epoch 7/100
13/13 - 0s - loss: 365.3225 - 14ms/epoch - 1ms/step
Epoch 8/100
13/13 - 0s - loss: 340.7126 - 15ms/epoch - 1ms/step
Epoch 9/100
13/13 - 0s - loss: 338.3392 - 21ms/epoch - 2ms/step
Epoch 10/100
13/13 - 0s - loss: 324.5045 - 16ms/epoch - 1ms/step
Epoch 11/100
13/13 - 0s - loss: 308.2317 - 20ms/epoch - 2ms/step
Epoch 12/100
13/13 - 0s - loss: 298.7678 - 17ms/epoch - 1ms/step
Epoch 13/100
13/13 - 0s - loss: 287.3769 - 15ms/epoch - 1ms/step
Epoch 14/100
13/13 - 0s - loss: 278.0823 - 19ms/epoch - 1ms/step
Epoch 15/100
13/13 - 0s - loss: 269.0443 - 21ms/epoch - 2ms/step
Epoch 16/100
13/13 - 0s - los

### Monitor the loss at each epoch

One line is produced for each training epoch.  You can eliminate this output by setting the verbose setting of the fit command:

* **verbose=0** - No progress output (use with Juputer if you do not want output)
* **verbose=1** - Display progress bar, does not work well with Jupyter
* **verbose=2** - Summary progress output (use with Jupyter if you want to know the loss at each epoch)

### Use Trained Model to Make Regression Prediction

Next we will perform actual predictions.  These predictions are assigned to the **pred** variable. These are all MPG predictions from the neural network.  

***Notice that the data to predict should be a 2D array!***  

***Notice that the prediction result is also a 2D array!***

Neural networks can return multiple values, so the result is always an array.  Here the neural network only returns 1 value per prediction (there are 398 cars, so 398 predictions).  However, a 2D array is needed because the neural network has the potential of returning more than one value.


In [ ]:
pred = model.predict(x)
print("Shape: {}".format(pred.shape))
print(pred)

13/13 [==============================] - 0s 1ms/step
Shape: (398, 1)
[[15.624652 ]
 [16.035051 ]
 [17.026726 ]
 [17.727373 ]
 [16.564383 ]
 [12.594571 ]
 [13.293249 ]
 [13.448194 ]
 [13.53947  ]
 [15.335877 ]
 [15.657725 ]
 [15.612835 ]
 [12.3518305]
 [20.13923  ]
 [27.01107  ]
 [20.66579  ]
 [21.088102 ]
 [21.093424 ]
 [28.053541 ]
 [27.231878 ]
 [25.407595 ]
 [26.298727 ]
 [27.733406 ]
 [28.262215 ]
 [21.022673 ]
 [15.827463 ]
 [17.759161 ]
 [17.862083 ]
 [16.285208 ]
 [28.489033 ]
 [25.650084 ]
 [27.93977  ]
 [29.17956  ]
 [20.813997 ]
 [18.035563 ]
 [17.32688  ]
 [16.46868  ]
 [18.132235 ]
 [14.035546 ]
 [11.820781 ]
 [13.606902 ]
 [14.642659 ]
 [10.383515 ]
 [10.126098 ]
 [ 8.597239 ]
 [19.260326 ]
 [24.211739 ]
 [17.45025  ]
 [17.055342 ]
 [25.814741 ]
 [27.837439 ]
 [28.858444 ]
 [28.013556 ]
 [30.195023 ]
 [31.06391  ]
 [28.503769 ]
 [28.24641  ]
 [28.453825 ]
 [27.68904  ]
 [27.348482 ]
 [26.24245  ]
 [26.762402 ]
 [14.150744 ]
 [12.754073 ]
 [14.99413  ]
 [14.057265 ]
 [17.31

We would like to see how good these predictions are. We know what the correct MPG is for each car, so we can measure how close the neural network was.

In [ ]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 3.9592444896698


We can also print out the first 10 cars, with predictions and actual MPG.

In [ ]:
# Sample predictions
for i in range(10):
    print("{}. Car name: {}, MPG: {}, predicted MPG: {}".format(i+1,cars[i],y[i],pred[i]))

1. Car name: chevrolet chevelle malibu, MPG: 18.0, predicted MPG: [15.624652]
2. Car name: buick skylark 320, MPG: 15.0, predicted MPG: [16.035051]
3. Car name: plymouth satellite, MPG: 18.0, predicted MPG: [17.026726]
4. Car name: amc rebel sst, MPG: 16.0, predicted MPG: [17.727373]
5. Car name: ford torino, MPG: 17.0, predicted MPG: [16.564383]
6. Car name: ford galaxie 500, MPG: 15.0, predicted MPG: [12.594571]
7. Car name: chevrolet impala, MPG: 14.0, predicted MPG: [13.293249]
8. Car name: plymouth fury iii, MPG: 14.0, predicted MPG: [13.448194]
9. Car name: pontiac catalina, MPG: 14.0, predicted MPG: [13.53947]
10. Car name: amc ambassador dpl, MPG: 15.0, predicted MPG: [15.335877]


## 2.3 Example of Tensorflow classification: Iris


This is a very simple example of how to perform the Iris classification using TensorFlow. The iris.csv file is used.

In [ ]:
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping


df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CSC219-MachineLearning/labs/data/iris.csv",na_values=['NA','?'])

species = encode_text_index(df,"species")

x,y = to_xy(df,"species")

In [ ]:
x

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [ ]:
x.shape

(150, 4)

In [ ]:
y  #  This is one-hot encoding.  Only one value is 1.0 (hot)

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

In [ ]:
y.shape

(150, 3)

In [ ]:
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(y.shape[1], activation='softmax')) # Output

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(x,y,verbose=2,epochs=100)

Epoch 1/100
5/5 - 2s - loss: 1.1507 - 2s/epoch - 370ms/step
Epoch 2/100
5/5 - 0s - loss: 0.9981 - 9ms/epoch - 2ms/step
Epoch 3/100
5/5 - 0s - loss: 0.9224 - 8ms/epoch - 2ms/step
Epoch 4/100
5/5 - 0s - loss: 0.8531 - 11ms/epoch - 2ms/step
Epoch 5/100
5/5 - 0s - loss: 0.7877 - 10ms/epoch - 2ms/step
Epoch 6/100
5/5 - 0s - loss: 0.7321 - 8ms/epoch - 2ms/step
Epoch 7/100
5/5 - 0s - loss: 0.6841 - 8ms/epoch - 2ms/step
Epoch 8/100
5/5 - 0s - loss: 0.6456 - 8ms/epoch - 2ms/step
Epoch 9/100
5/5 - 0s - loss: 0.6094 - 8ms/epoch - 2ms/step
Epoch 10/100
5/5 - 0s - loss: 0.5814 - 8ms/epoch - 2ms/step
Epoch 11/100
5/5 - 0s - loss: 0.5531 - 8ms/epoch - 2ms/step
Epoch 12/100
5/5 - 0s - loss: 0.5289 - 10ms/epoch - 2ms/step
Epoch 13/100
5/5 - 0s - loss: 0.5089 - 9ms/epoch - 2ms/step
Epoch 14/100
5/5 - 0s - loss: 0.4923 - 10ms/epoch - 2ms/step
Epoch 15/100
5/5 - 0s - loss: 0.4755 - 10ms/epoch - 2ms/step
Epoch 16/100
5/5 - 0s - loss: 0.4649 - 9ms/epoch - 2ms/step
Epoch 17/100
5/5 - 0s - loss: 0.4562 - 10ms

In [ ]:
# Print out number of species found:
print(species)

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


Now that you have a neural network trained, we would like to be able to use it. There were 3 types of iris (Iris-setosa, Iris-versicolor, and Iris-virginica).

In [ ]:
pred = model.predict(x)
pred.shape

5/5 [==============================] - 0s 2ms/step


(150, 3)

In [ ]:
pred

array([[9.97788072e-01, 2.21190299e-03, 4.45829151e-09],
       [9.95578945e-01, 4.42100735e-03, 2.47375329e-08],
       [9.96394813e-01, 3.60524911e-03, 1.88487608e-08],
       [9.93545234e-01, 6.45466708e-03, 6.09107857e-08],
       [9.97823358e-01, 2.17669317e-03, 4.54204141e-09],
       [9.97098446e-01, 2.90157506e-03, 4.70305528e-09],
       [9.95499551e-01, 4.50059772e-03, 2.99236937e-08],
       [9.96895134e-01, 3.10490723e-03, 9.72257741e-09],
       [9.92188394e-01, 7.81141128e-03, 1.18226474e-07],
       [9.96010363e-01, 3.98970861e-03, 1.82260607e-08],
       [9.98367608e-01, 1.63238018e-03, 1.72214665e-09],
       [9.95293021e-01, 4.70701652e-03, 2.40894060e-08],
       [9.95935798e-01, 4.06413060e-03, 2.19911875e-08],
       [9.96085942e-01, 3.91401676e-03, 3.27211858e-08],
       [9.99406576e-01, 5.93426405e-04, 1.35618072e-10],
       [9.99112368e-01, 8.87518981e-04, 3.27470995e-10],
       [9.98601258e-01, 1.39873882e-03, 1.29460420e-09],
       [9.97366428e-01, 2.63348

In [ ]:
#print y
print(y)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0.

The column (pred) with the highest probability is the prediction of the neural network.
Use argmax function to find the index of the maximum prediction for each row.

In [ ]:
# Usually the column (pred) with the highest prediction is considered to be the prediction of the neural network.  It is easy
# to convert the predictions to the expected iris species.  The argmax function finds the index of the maximum prediction
# for each row.

predict_classes = np.argmax(pred,axis=1)

true_classes = np.argmax(y,axis=1)

print("Predictions: {}".format(predict_classes))
print("True: {}".format(true_classes))

Predictions: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
True: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [ ]:
# Of course it is very easy to turn these indexes back into iris species.  We just use the species list that we created earlier.
print(species[predict_classes[0:10]])

['Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa']


In [ ]:
#For all of the iris predictions, what percent were correct?

correct = metrics.accuracy_score(true_classes, predict_classes)
print("Accuracy: {}".format(correct))

Accuracy: 0.98


The code below performs two ad hoc predictions.

*** Remember x should be a 2D array! ***

In [ ]:
# ad hoc prediction
sample_flower = np.array( [[5.0,3.0,4.0]], dtype=float)
pred = np.argmax(pred)
print("Predict that {} is: {}".format(sample_flower,species[pred]))

IndexError: ignored

In [ ]:
# predict two sample flowers
sample_flower = np.array( [[5.0,3.0,4.0,2.0],[5.2,3.5,1.5,0.8]], dtype=float)
pred = np.argmax(pred)
print("Predict that {} is: {}".format(sample_flower,species[pred]))

Predict that [[5.  3.  4.  2. ]
 [5.2 3.5 1.5 0.8]] is: Iris-setosa


### Load/Save Trained Network


Complex neural networks will take a long time to fit/train.  It is helpful to be able to save these neural networks so that they can be reloaded later.  A reloaded neural network will not require retraining.  Keras provides three formats for neural network saving.

* **YAML** - Stores the neural network structure (no weights) in the [YAML file format](https://en.wikipedia.org/wiki/YAML).
* **JSON** - Stores the neural network structure (no weights) in the [JSON file format](https://en.wikipedia.org/wiki/JSON).
* **HDF5** - Stores the complete neural network (with weights) in the [HDF5 file format](https://en.wikipedia.org/wiki/Hierarchical_Data_Format).

Usually you will want to save in HDF5.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
path = "/content/drive/MyDrive/Colab Notebooks/CSC219-MachineLearning/labs/data"
save_path = "/content/drive/MyDrive/Colab Notebooks/CSC219-MachineLearning/labs/data"

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

cars = df['name']
df.drop('name',1,inplace=True)
missing_median(df, 'horsepower')
x,y = to_xy(df,"mpg")
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x,y,verbose=0,epochs=100)

# Predict
pred = model.predict(x)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print("Before save score (RMSE): {}".format(score))


# save entire network to HDF5 (save everything)
model.save(os.path.join(save_path,"network.hdf5"))

<ipython-input-39-a9c16b4fcf0e>:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop('name',1,inplace=True)


13/13 [==============================] - 0s 1ms/step
Before save score (RMSE): 4.95494270324707


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Now we reload the network and perform another prediction. The RMSE should match the previous one exactly if the neural network was really saved and reloaded.

In [ ]:
from tensorflow.keras.models import load_model

model2 = load_model(os.path.join(save_path,"network.hdf5"))
pred = model2.predict(x)
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print("After load score (RMSE): {}".format(score))

13/13 [==============================] - 0s 1ms/step
After load score (RMSE): 4.95494270324707


## References:



* [Google Colab](https://colab.research.google.com/) - Free web based platform that includes Python, Juypter Notebooks, and TensorFlow with free GPU support.  No setup needed.
* [IBM Cognitive Class Labs](https://www.datascientistworkbench.com) - Free web based platform that includes Python, Juypter Notebooks, and TensorFlow.  No setup needed.
* [Python Anaconda](https://www.continuum.io/downloads) - Python distribution that includes many data science packages, such as Numpy, Scipy, Scikit-Learn, Pandas, and much more.
* [TensorFlow](https://www.tensorflow.org/) - Google's mathematics package for deep learning.
* [Kaggle](https://www.kaggle.com/) - Competitive data science.  Good source of sample data.
* T81-558: Applications of Deep Neural Networks. Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/)